# 🔍 Анализ преступности в регионах России (2011–2022)
В этом проекте проводится исследование преступности по федеральным округам и регионам России с использованием кластеризации и PCA. Также выявляются наиболее часто встречающиеся статьи УК РФ в разных кластерах.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Загрузка данных
df = pd.read_excel('data_crime_107_v20231202_long.xlsx')
df.shape

In [ ]:
# Подготовка данных по федеральным округам
okrug_df = df[(df['object_level'] == 'федеральный округ') &
              (df['indicator_unit'] == 'единиц') &
              (df['year'].between(2011, 2022))]
target_indicators = [
    'Не раскрыто',
    'Предварительно расследовано',
    'Зарегистрировано',
    'дела о которых направлены в суд'
]
okrug_df = okrug_df[okrug_df['indicator_name'].str.contains('|'.join(target_indicators))]
okrug_df['indicator_year'] = okrug_df.apply(
    lambda row: f"{[k for k in target_indicators if k in row['indicator_name']][0]}_{row['year']}", axis=1
)
stat_okrug = okrug_df.pivot_table(index='object_name',
                                  columns='indicator_year',
                                  values='indicator_value',
                                  aggfunc='sum').fillna(0)
stat_okrug.head()

In [ ]:
# Кластеризация федеральных округов
scaler = StandardScaler()
X_scaled = scaler.fit_transform(stat_okrug)
kmeans = KMeans(n_clusters=3, random_state=42)
okrug_labels = kmeans.fit_predict(X_scaled)
stat_okrug['cluster'] = okrug_labels
stat_okrug[['cluster']].sort_values('cluster')

In [ ]:
# Визуализация временных рядов
stat_okrug.drop(columns='cluster').T.plot(figsize=(12, 6))
plt.title('Динамика по всем округам')
plt.ylabel('Количество преступлений')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Работа с данными по статьям УК РФ
region_df = df[(df['object_level'] == 'регион') &
               (df['indicator_unit'] == 'единиц') &
               (df['year'].between(2011, 2022))]
region_df = region_df[region_df['indicator_name'].str.contains('|'.join(target_indicators))]
region_df['indicator_year'] = region_df.apply(
    lambda row: f"{[k for k in target_indicators if k in row['indicator_name']][0]}_{row['year']}", axis=1
)
stat_region = region_df.pivot_table(index='object_name',
                                    columns='indicator_year',
                                    values='indicator_value',
                                    aggfunc='sum').fillna(0)
X_region_scaled = scaler.fit_transform(stat_region)
region_labels = kmeans.fit_predict(X_region_scaled)
stat_region['cluster'] = region_labels
normalized_region_names = stat_region.index.str.lower().str.replace('г. ', '').str.strip()

In [ ]:
# Выявление статей УК РФ по кластерам
article_df = df[(df['object_level'] == 'регион') &
    (df['indicator_unit'] == 'единиц') &
    (df['indicator_name'].str.contains(r'ст\.', case=False, na=False))].copy()
article_df['normalized_name'] = article_df['object_name'].str.lower().str.replace('г. ', '').str.strip()
mapping = dict(zip(normalized_region_names, stat_region['cluster']))
article_df['cluster'] = article_df['normalized_name'].map(mapping)
article_df = article_df[~article_df['cluster'].isna()]
top_articles_by_cluster = (
    article_df.groupby('cluster')['indicator_name']
    .apply(lambda x: x.value_counts().head(5))
)
print(top_articles_by_cluster)

## 💡 Выводы
- Выделены кластеры округов и регионов по уровню преступности
- Определены регионы с положительной динамикой
- Выявлены наиболее частые статьи УК РФ внутри каждого кластера